In [17]:
from transformers import DistilBertForMaskedLM, DistilBertTokenizerFast
from transformers import PretrainedConfig
from tqdm import tqdm
import numpy as np

%load_ext autoreload
%autoreload 2

model = DistilBertForMaskedLM.from_pretrained("./model_dbertft/", from_tf=True)

tokenizer = DistilBertTokenizerFast.from_pretrained("bert-base-uncased")

You passed along `num_labels=1200` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


All TF 2.0 model weights were used when initializing DistilBertForMaskedLM.

Some weights of DistilBertForMaskedLM were not initialized from the TF 2.0 model and are newly initialized: ['vocab_projector.weight', 'vocab_projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizerFast'.


In [2]:
# text = "The capital of France, {} , contains the Eiffel Tower, a very popular tourist attraction."

# vector_input = tokenizer.encode(text, return_tensors="pt")

# probabilities = model(vector_input)[0]

In [3]:
from utils import get_news_list, get_users_list

news = get_news_list()
users = get_users_list()

users_dict = {}
for user in users:
    users_dict[user.user_id] = user

Loading users from mongodb...


100%|██████████| 500/500 [00:04<00:00, 108.64it/s]


Loading news from mongodb...


100%|██████████| 24586/24586 [00:00<00:00, 37527.92it/s]


Loading users from mongodb...


100%|██████████| 500/500 [00:04<00:00, 113.80it/s]


In [ ]:
from models import get_cosine_similarity, get_user_embeddings, get_news_embeddings

In [6]:
historical_embeddings, current_embeddings = get_user_embeddings(news[:30], news[0])

get_cosine_similarity(historical_embeddings[0], current_embeddings[0])

0.37207454104562

In [9]:
#nDCG25

In [10]:
import joblib

test_batches = joblib.load("test_batches.pkl")

In [15]:
for user in test_batches.keys():
    print(users_dict[user].train_list)
    break

['The Broward County Sheriff’s Office late Thursday released the names of the 17 people killed in the mass shooting at Marjory Stoneman Douglas High School on Wednesday. Earlier Thursday, relatives and friends of the victims took to social media to mourn and honor the people they lost. The victims were:\nAlyssa Alhadeff, 14\nNever miss a local story. Sign up today for a free 30 day free trial of unlimited digital access. SUBSCRIBE NOW\nAlyssa, a freshman from Parkland, had been reported missing by her grandmother Wednesday after the shooting. Her cousin Ariella Del Quaglio confirmed the student’s death via Facebook on Thursday.\nScott Beigel, 35\nScott Beigel, 35\nBeigel was a geography teacher and a cross country coach at the school who unlocked his classroom to allow panicked students to take shelter inside during the shooting. The Coral Springs man was struck and killed by a bullet while closing the door behind them.\nKelsey Friend, one of the students whose lives he saved, told CNN

In [16]:
import joblib

final_results = {}
with tqdm(total=len(test_batches)) as pbar:
    for user_key, batch in test_batches.items():
        pbar.update(1)
        test_batch_news_text, test_batch_isread, test_batch_key = batch["news_texts"] , batch["news_is_relevant"], batch["key"]
        historical_embeddings = None
        scores = []
        historical_news = users_dict[user_key].train_list[:5]
        for news in test_batch_news_text:
            current_news = news
            historical_embeddings, current_embeddings = get_user_embeddings(historical_news, current_news, historical_embeddings)
            scores.append(get_cosine_similarity(historical_embeddings[0], current_embeddings[0]))
        scores = np.array(scores)
        final_results[user_key] = [(key, score, is_relevant) for key, score, is_relevant in zip(test_batch_key, scores, test_batch_isread)]
        joblib.dump(final_results, "ranking_dbertft.pkl")


100%|██████████| 500/500 [11:39:32<00:00, 83.95s/it]   


In [88]:
from sklearn.metrics import ndcg_score

ndcg_score(is_relevant.reshape(1, -1), scores.reshape(1, -1))

0.6496804427420483